### Read data file from online JSON file

In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
import requests
import datetime as dt
import matplotlib.dates as mdates
plt.rcParams['figure.dpi'] = 100

# set conditions
countryCode = "FRA"
key = "new_cases"
blue = "#1f77b4"
orange = "#ff7f0e"
diffDayMax = +50
diffDayMin = -50

# read JSON file from URL
data = requests.get("https://covid.ourworldindata.org/data/owid-covid-data.json").json()
#f = open('owid-covid-data.json', 'r')
#data = json.load(f)
country_data = data[countryCode]["data"]
countryName = data[countryCode]["location"]

# show sample key list
print("Key list:")
for sample_key in country_data[-2]:
    print(sample_key)

### Show time-series data

In [ ]:
dates = []
value = []

for datedata in country_data:
    t = datedata.get(key, "nan") # if there is no key, return nan
    
    if(t != "nan"):
        value.append(float(t)) # add to the list with casting the value from string to float
        dates.append(datedata.get("date"))

# convert into datatime format
dates = [dt.datetime.strptime(d, '%Y-%m-%d').date() for d in dates]

# visualize
fig, ax = plt.subplots()
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y/%m'))
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=120))

ax.set_title(countryName)
ax.plot(dates, value, blue)
plt.gcf().autofmt_xdate()
ax.set_ylabel(key)
plt.show()

### Show correlation

In [ ]:
# compute mean and standard deviation 
v_var  = np.var(value)
v_mean = np.mean(value)

numDates = len(dates)
maxDiffDay = (dates[-1] - dates[0]).days + 1
correlSize = 2*(maxDiffDay - 1) + 1
diffs = np.arange(- (maxDiffDay - 1), maxDiffDay)
correlations = np.zeros([correlSize])
counts = np.zeros([correlSize])

# compute correlation
for i_day in range(numDates):
    for j_day in range(numDates):
        diffDay = (dates[j_day] - dates[i_day]).days + (maxDiffDay - 1)

        correlations[diffDay] += (value[i_day] - v_mean)*(value[j_day] - v_mean)
        counts[diffDay] += 1

correlations /= v_var*counts

# get information of maximum correlation
diffs = np.array(diffs)
correlations = np.array(correlations)
cliped_diffs = diffs[(diffs > diffDayMin) & (diffs < diffDayMax)]
cliped_correlations = correlations[(diffs > diffDayMin) & (diffs < diffDayMax)]
max_day    = cliped_diffs[cliped_correlations.argmax()]
max_correl = cliped_correlations[cliped_correlations.argmax()]

print("----- Max correlation:", max_day, max_correl)

# visualize
fig, ax = plt.subplots()
ax.set_title(countryName)
ax.plot(diffs, correlations)
ax.plot(max_day, max_correl, "ro")
ax.set_xlabel('diff_day')
ax.set_ylabel('correlation')
ax.set_xlim([diffDayMin, diffDayMax])
plt.show()